# In the name of Allah

In [2]:
!pip install networkx numpy

import networkx as nx
import numpy as np
from collections import deque


# Graph 15 nodes

In [5]:

def build_nx_graph_15():
    G = nx.DiGraph()
    G.add_nodes_from(range(15))

    edges = [
        (0,1,10),(0,2,8),(1,3,5),(2,3,6),(1,4,7),(2,5,7),
        (3,6,10),(4,6,6),(5,6,6),(4,7,4),(5,8,4),
        (6,9,12),(7,9,5),(8,9,5),(9,10,10),
        (3,11,3),(11,12,4),(12,13,6),(13,14,8),
        (10,7,2),(8,5,2),(5,2,1),(7,4,1)
    ]

    for u, v, c in edges:
        G.add_edge(u, v, capacity=float(c))
        if not G.has_edge(v, u):
            G.add_edge(v, u, capacity=float(0.5 * c))

    return G

# تست سریع
G = build_nx_graph_15()
print("Graph created with", G.number_of_nodes(), "nodes and", G.number_of_edges(), "edges.")


Graph created with 15 nodes and 40 edges.


## تبدیل گراف به ماتریس ظرفیت

In [7]:

def nx_to_capacity_matrix(G: nx.DiGraph) -> np.ndarray:
    n = G.number_of_nodes()
    C = np.zeros((n, n), dtype=float)
    for u, v, data in G.edges(data=True):
        C[u, v] = data.get("capacity", 0.0)
    return C

C = nx_to_capacity_matrix(G)
print("Capacity matrix shape:", C.shape)


Capacity matrix shape: (15, 15)


## Edmonds–Karp Algo

In [8]:
# =========================
# 📦 Cell 4: الگوریتم
# =========================

def bfs_augmenting_path(cap: np.ndarray, flow: np.ndarray, s: int, t: int):
    n = cap.shape[0]
    parent = [-1] * n
    parent[s] = s
    q = deque([s])

    while q:
        u = q.popleft()
        for v in range(n):
            residual = cap[u, v] - flow[u, v]
            if residual > 1e-6 and parent[v] == -1:
                parent[v] = u
                if v == t:
                    path = []
                    bottleneck = float('inf')
                    cur = t
                    while cur != s:
                        prev = parent[cur]
                        path.append((prev, cur))
                        bottleneck = min(bottleneck, cap[prev, cur] - flow[prev, cur])
                        cur = prev
                    path.reverse()
                    return path, bottleneck
                q.append(v)
    return [], 0.0

def edmonds_karp_max_flow(cap: np.ndarray, s: int, t: int):
    n = cap.shape[0]
    flow = np.zeros((n, n), dtype=float)
    maxflow = 0.0

    while True:
        path, bottleneck = bfs_augmenting_path(cap, flow, s, t)
        if bottleneck == 0.0:
            break
        maxflow += bottleneck
        for u, v in path:
            flow[u, v] += bottleneck
            flow[v, u] -= bottleneck
    return maxflow, flow

def min_cut_from_flow(cap: np.ndarray, flow: np.ndarray, s: int):
    n = cap.shape[0]
    visited = [False] * n
    stack = [s]
    visited[s] = True

    while stack:
        u = stack.pop()
        for v in range(n):
            residual = cap[u, v] - flow[u, v]
            if residual > 1e-6 and not visited[v]:
                visited[v] = True
                stack.append(v)

    S = [i for i in range(n) if visited[i]]
    T = [i for i in range(n) if not visited[i]]
    return S, T


## Final Test

In [9]:
source, sink = 0, 10
maxflow, flow = edmonds_karp_max_flow(C, s=source, t=sink)
print(f"✅ Max flow from {source} to {sink}:", maxflow)

S, T = min_cut_from_flow(C, flow, s=source)
print("✅ Min-cut partition:")
print("S =", S)
print("T =", T)

✅ Max flow from 0 to 10: 11.0
✅ Min-cut partition:
S = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14]
T = [10]
